## Working with GeoJSON

In the 3.0 release of pandapower, significant changes have been made to how geospatial data is stored in the net object. Previously, geospatial data was stored in the *line_geodata* and *bus_geodata* tables. However, this structure caused several problems:

- In what projection are the data points stored?
- What do x and y represent? Does x represent latitude, longitude or something else?
- Copmatibility issues with frontend services and QGIS

To resolve this ambiguity, all geospatial data has now been moved to the line and bus tables under the *geo* column. The geometries are stored as GeoJSON objects. [GeoJSON](https://datatracker.ietf.org/doc/html/rfc7946) is a widely used standard for geospatial data, supporting points, lines, and polygons in a unified format with explicit projection definitions.

In [ ]:
from pandapower.networks import mv_oberrhein
import pandas as pd
pd.set_option("display.max_colwidth", 120)

In [ ]:
net = mv_oberrhein()
print(net.line.geo.head(), "\n")
print(net.bus.geo.head())

To make working with these new geospatial data easier and more efficient, a pandas series accessor has been introduced. This accessor allows direct interaction with GeoJSON data and integrates functions provided by GeoSeries in geopandas.

In [ ]:
from shapely.geometry import Point
from geopandas import GeoSeries
from pandapower.plotting import simple_plotly, create_line_trace, create_bus_trace
import plotly.graph_objects as go

Create a reference point and a polygon to check what elements are inside a certain radius. 

In [ ]:
reference_point = (7.781067, 48.389774)
radius_m = 2200
circle_polygon = GeoSeries([Point(reference_point)], crs=4326).to_crs(epsg=31467).buffer(radius_m).to_crs(epsg=4326).iloc[0]

Access the GeoJSON accessor and the needed GeoSeries methods.

In [ ]:
lines_intersect = net.line[net.line.geo.geojson.intersects(circle_polygon)].index
buses_within = net.bus[net.bus.geo.geojson.within(circle_polygon)].index
net.line.geo.geojson.total_bounds

And plot the results

In [ ]:
x, y = circle_polygon.exterior.xy
polygon_trace = go.Scatter(
    x=list(x), 
    y=list(y), 
    fill="toself",
    mode="lines",
    line={"color": "orange", "width": 0.5},
    fillcolor="rgba(255, 165, 0, 0.2)",
    name="radius"
)

point_trace = go.Scatter(
    x=[circle_polygon.centroid.x], 
    y=[circle_polygon.centroid.y], 
    mode="markers",
    marker={"color": "orange", "size": 10},
    name="reference point"
)
lt = create_line_trace(net, lines=lines_intersect, color="red", trace_name='intersecting lines')
bt = create_bus_trace(net, buses=buses_within, color="red", trace_name="buses within radius")
fig = simple_plotly(net, auto_open=False, additional_traces=lt)
fig.add_trace(polygon_trace)
fig.add_trace(point_trace)
fig.add_trace(bt[0])

It is also possible to load the GeoJSON, or view the geo column as GeoSeries or a Series with shapely objects.

In [ ]:
net.bus.geo.head() # entries are strings

In [ ]:
net.bus.geo.geojson.as_geo_obj.head() # entries are dicts

In [ ]:
net.bus.geo.geojson.as_geoseries.head() # GeoSeries 

In [ ]:
net.bus.geo.geojson.as_shapely_obj.head() # pandas Series with shapely objects

It is also possible to get the coordinates or the geometry type from the GeoJSON. 
**Note that it is not recommended to use ._coords in applications, because the projection definition is lost!**

In [ ]:
net.bus.geo.geojson._coords.head()

In [ ]:
net.bus.geo.geojson.type.head()